In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
import joblib

# Load dataset
file_path = "gym recommendation.xlsx"
df = pd.read_excel(file_path, sheet_name='Sheet1')

# Drop unnecessary columns
df = df.drop(columns=['ID', 'Fitness Type'])

# Ensure 'Fitness Goal' has only 'Weight Loss' and 'Weight Gain'
df = df[df['Fitness Goal'].isin(['Weight Loss', 'Weight Gain'])]

# Encoding categorical variables
label_encoders = {}
categorical_columns = ['Sex', 'Hypertension', 'Diabetes', 'Level', 'Fitness Goal', 'Exercises', 'Equipment', 'Diet', 'Recommendation']
for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))
    label_encoders[col] = le

# Splitting data into input (X) and output (y)
X = df[['Sex', 'Age', 'Height', 'Weight', 'Hypertension', 'Diabetes', 'BMI', 'Fitness Goal']]
y = df[['Level', 'Exercises', 'Equipment', 'Diet', 'Recommendation']]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Model pipeline
pipeline = Pipeline([
    ('classifier', MultiOutputClassifier(RandomForestClassifier(n_estimators=100, random_state=42)))
])

# Train model
pipeline.fit(X_train, y_train)

# Save the model and encoders
joblib.dump(pipeline, "nlp_fitness_model3.pkl")
joblib.dump(label_encoders, "label_encoders3.pkl")

print("Model updated and saved successfully!")

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import joblib

# Load dataset
file_path = "gym recommendation.xlsx"
df = pd.read_excel(file_path, sheet_name='Sheet1')

# Drop unnecessary columns
df = df.drop(columns=['ID', 'Fitness Type'])

# Ensure 'Fitness Goal' has only 'Weight Loss' and 'Weight Gain'
df = df[df['Fitness Goal'].isin(['Weight Loss', 'Weight Gain'])].copy()

# Feature Engineering: Activity Level
def assign_activity_level(row):
    if row["BMI"] < 18.5 and row["Age"] < 30:
        return "Very Active"
    elif 18.5 <= row["BMI"] < 25:
        return "Moderately Active"
    elif 25 <= row["BMI"] < 30 or row["Age"] >= 40:
        return "Lightly Active"
    else:
        return "Sedentary"

df["Activity_Level"] = df.apply(assign_activity_level, axis=1)

# Experience Level
def assign_experience_level(row):
    if row["Activity_Level"] in ["Moderately Active", "Very Active"] and row["BMI"] < 25:
        return "Advanced"
    elif row["Activity_Level"] == "Lightly Active" or (row["BMI"] >= 25 and row["BMI"] < 30):
        return "Intermediate"
    else:
        return "Beginner"

df["Experience_Level"] = df.apply(assign_experience_level, axis=1)

# Encode categorical variables
label_encoders = {}
categorical_columns = ['Sex', 'Hypertension', 'Diabetes', 'Fitness Goal', 'Level', 'Exercises', 'Equipment', 'Diet', 'Recommendation', 'Experience_Level', 'Activity_Level']

for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))
    label_encoders[col] = le  # Save encoders for later decoding

# Feature Selection
X = df[['Sex', 'Age', 'Height', 'Weight', 'BMI', 'Hypertension', 'Diabetes', 'Fitness Goal', 'Experience_Level', 'Activity_Level']]
y = df[['Level', 'Exercises', 'Equipment', 'Diet', 'Recommendation']]

# Standardize numerical features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# Convert target labels to NumPy arrays
y_train = [y_train[col].values for col in y.columns]
y_test = [y_test[col].values for col in y.columns]

# Model Architecture
input_layer = Input(shape=(X_train.shape[1],))

hidden = Dense(64, activation='relu')(input_layer)
hidden = BatchNormalization()(hidden)
hidden = Dropout(0.3)(hidden)
hidden = Dense(128, activation='relu')(hidden)
hidden = BatchNormalization()(hidden)
hidden = Dropout(0.3)(hidden)

# Output layers for multi-task learning
outputs = [Dense(len(df[col].unique()), activation='softmax', name=col)(hidden) for col in y.columns]

# Build Model
model = Model(inputs=input_layer, outputs=outputs)
# Compile model with separate loss and accuracy metrics for each output
model.compile(
    optimizer='adam',
    loss=['sparse_categorical_crossentropy'] * len(y.columns),
    metrics=[['accuracy']] * len(y.columns)
)

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=16, verbose=1)

# Save model and encoders
model.save("deep_fitness_recommender.h5")
joblib.dump(label_encoders, "label_encoderss.pkl")
joblib.dump(scaler, "scalers.pkl")

print("Deep learning model trained and saved successfully!")


Epoch 1/100
639/639 [==============================] - 3s 2ms/step - loss: 3.3520 - Level_loss: 0.4068 - Exercises_loss: 0.2886 - Equipment_loss: 0.8863 - Diet_loss: 0.9477 - Recommendation_loss: 0.8227 - Level_accuracy: 0.8404 - Exercises_accuracy: 0.9054 - Equipment_accuracy: 0.7484 - Diet_accuracy: 0.7445 - Recommendation_accuracy: 0.7875 - val_loss: 1.4705 - val_Level_loss: 0.0722 - val_Exercises_loss: 0.0260 - val_Equipment_loss: 0.4756 - val_Diet_loss: 0.4529 - val_Recommendation_loss: 0.4438 - val_Level_accuracy: 0.9829 - val_Exercises_accuracy: 0.9966 - val_Equipment_accuracy: 0.8878 - val_Diet_accuracy: 0.9141 - val_Recommendation_accuracy: 0.9168
Epoch 2/100
639/639 [==============================] - 1s 2ms/step - loss: 1.8663 - Level_loss: 0.1848 - Exercises_loss: 0.0713 - Equipment_loss: 0.5719 - Diet_loss: 0.5292 - Recommendation_loss: 0.5091 - Level_accuracy: 0.9299 - Exercises_accuracy: 0.9846 - Equipment_accuracy: 0.8521 - Diet_accuracy: 0.8837 - Recommendation_accuracy

In [3]:
import joblib

# Load the label encoders
label_encoders = joblib.load("label_encoderss.pkl")

# Display encoded classes for categorical features
print("Categorical Features:\n")
for feature, encoder in label_encoders.items():
    print(f"Feature: {feature}")
    print(f"Classes: {encoder.classes_}\n")

# Display unique values for numerical features directly from the DataFrame
numerical_columns = ['Age', 'Height', 'Weight', 'BMI', 'Experience_Level', 'Activity_Level']

print("Numerical Features:\n")
for feature in numerical_columns:
    unique_values = df[feature].unique()
    print(f"Feature: {feature}")
    print(f"Unique Values: {np.sort(unique_values)}\n")


Categorical Features:

Feature: Sex
Classes: ['Female' 'Male']

Feature: Hypertension
Classes: ['No' 'Yes']

Feature: Diabetes
Classes: ['No' 'Yes']

Feature: Fitness Goal
Classes: ['Weight Gain' 'Weight Loss']

Feature: Level
Classes: ['Normal' 'Obuse' 'Overweight' 'Underweight']

Feature: Exercises
Classes: ['Brisk walking, cycling, swimming, running , or dancing.'
 'Squats, deadlifts, bench presses, and overhead presses'
 'Squats, yoga, deadlifts, bench presses, and overhead presses'
 'Walking, Yoga, Swimming.'
 'brisk walking, cycling, swimming, or dancing.']

Feature: Equipment
Classes: ['Dumbbells and barbells' 'Dumbbells, barbells and Blood glucose monitor'
 'Ellipticals, Indoor Rowers,Treadmills, Rowing machine'
 'Ellipticals, Indoor Rowers,Treadmills, and Rowing machine'
 'Equipment Required' 'Kettlebell, Dumbbells, Yoga Mat'
 'Kettlebell, Dumbbells, Yoga Mat, Treadmill'
 'Light athletic shoes, resistance bands, and light dumbbells.'
 'Light athletic shoes, resistance bands, l

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
import joblib

# Load dataset
file_path = "gym recommendation.xlsx"
df = pd.read_excel(file_path, sheet_name='Sheet1')

# Drop unnecessary columns
df = df.drop(columns=['ID', 'Fitness Type'])

# Ensure 'Fitness Goal' has only 'Weight Loss' and 'Weight Gain'
df = df[df['Fitness Goal'].isin(['Weight Loss', 'Weight Gain'])].copy()  # Use .copy() to avoid SettingWithCopyWarning

# Feature Engineering: Activity Level
def assign_activity_level(row):
    if row["BMI"] < 18.5 and row["Age"] < 30:
        return "Very Active"
    elif 18.5 <= row["BMI"] < 25:
        return "Moderately Active"
    elif 25 <= row["BMI"] < 30 or row["Age"] >= 40:
        return "Lightly Active"
    else:
        return "Sedentary"

df["Activity_Level"] = df.apply(assign_activity_level, axis=1)

# Feature Engineering: Experience Level
def assign_experience_level(row):
    if row["Activity_Level"] in ["Moderately Active", "Very Active"] and row["BMI"] < 25:
        return "Advanced"
    elif row["Activity_Level"] == "Lightly Active" or (row["BMI"] >= 25 and row["BMI"] < 30):
        return "Intermediate"
    else:
        return "Beginner"

df["Experience_Level"] = df.apply(assign_experience_level, axis=1)

# Encoding categorical variables
label_encoders = {}
categorical_columns = ['Sex', 'Hypertension', 'Diabetes', 'Level', 'Fitness Goal', 'Exercises', 
                       'Equipment', 'Diet', 'Recommendation', 'Activity_Level', 'Experience_Level']
for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))
    label_encoders[col] = le

# Splitting data into input (X) and output (y)
X = df[['Sex', 'Age', 'Height', 'Weight', 'Hypertension', 'Diabetes', 'BMI', 'Fitness Goal', 
        'Activity_Level', 'Experience_Level']]
y = df[['Level', 'Exercises', 'Equipment', 'Diet', 'Recommendation']]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Model pipeline
pipeline = Pipeline([
    ('classifier', MultiOutputClassifier(RandomForestClassifier(n_estimators=100, random_state=42)))
])

# Train model
pipeline.fit(X_train, y_train)

# Save the model and encoders
joblib.dump(pipeline, "nlp_fitness_model3.pkl")
joblib.dump(label_encoders, "label_encoders3.pkl")

print("Model updated and saved successfully!")

Model updated and saved successfully!


In [5]:
# Example input with all features
sample = pd.DataFrame([[1, 25, 170, 70, 0, 0, 24.2, 0, 1, 1]], 
                      columns=['Sex', 'Age', 'Height', 'Weight', 'Hypertension', 'Diabetes', 
                               'BMI', 'Fitness Goal', 'Activity_Level', 'Experience_Level'])
pred = pipeline.predict(sample)
for i, col in enumerate(y.columns):
    pred_label = label_encoders[col].inverse_transform(pred[:, i])[0]
    print(f"{col}: {pred_label}")

Level: Normal
Exercises: Squats, deadlifts, bench presses, and overhead presses
Equipment: Dumbbells and barbells
Diet: Vegetables: (Mixed greens, cherry tomatoes, cucumbers, bell peppers, carrots, celery, bell peppers);Protein Intake: (Chicken, fish, tofu, or legumes); Juice : (Green juice,kale, spinach, cucumber, celery, and apple)
Recommendation: Follow a regular exercise schedule. Adhere to the exercise and diet plan to get better results. It is important to approach weight gain in a healthy and balanced way, focusing on both exercise and nutrition. Keep in mind that weight gain should be gradual and focused on building lean muscle rather than increasing fat. Additionally, it's always a good idea to consult with a health care professional or registered dietitian before making any significant changes to your exercise or diet plan. Here are some important tips:- Stay hydrated by drinking enough water throughout the day. Monitor your progress and adjust your diet and exercise routine 

In [6]:
from sklearn.metrics import accuracy_score
y_pred = pipeline.predict(X_test)
for i, col in enumerate(y.columns):
    print(f"{col} accuracy: {accuracy_score(y_test[col], y_pred[:, i])}")

Level accuracy: 1.0
Exercises accuracy: 0.9984007310943569
Equipment accuracy: 0.9453963902216129
Diet accuracy: 0.9481379940598583
Recommendation accuracy: 0.9527073337902673
